# #0 Table of Content
1. Import packages and data

# #1 Import Packages and data


In [130]:
import pandas as pd
import uuid

In [117]:
product_table = pd.read_csv("../asset/product.csv")
review_table  = pd.read_csv("../asset/review.csv")
user_table    = pd.read_csv("../asset/user.csv")

# #2 Data Correction

## #2.1 Product Table
1. Drop 'Unnamed: 0' column.
2. Rename column names according to the Data Dictionary.
3. Ensure each column has the correct data type according to the Data Dictionary.

In [118]:
# PRODUCT
print(product_table.shape)
display(product_table.head(3))
print(product_table.info())

(75, 11)


,Unnamed: 0,Product ID:,Store Name,Product Name:,Product Price:,Total Number of Ratings:,Average Ratings:,Total Sold:,Total Fav:,Total Available:,Description:
0,0,1ca606d9-0507-410b-a98c-a0d1745cc233,Skullcandy Singapore Official Store,Crusher ANC 2 Sensory Bass Wireless Headphones...,$350.00,3,5.0,7,Favorite (43),2 pieces available,The next generation of our iconic Crusher head...
1,1,a45e9ef8-c2d2-4495-ac16-0b8c42a236ca,sprise_localstore.sg,SPRISE Premium Wireless Bluetooth Earphone Col...,$17.33,245,4.6,813,Favorite (1.1k),2 pieces available,Hey~ Welcome to SPRISE Official Store! Please ...
2,2,cdba870d-eb79-4380-9de9-d6e61c895949,SHAVALIFE,USB PC Headset Conference Call with Noise Canc...,$18.50,1.6k,4.9,5.1k,Favorite (1.5k),477 pieces available,Headset Key Features: ✅ High Quality Stereo S...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                75 non-null     int64  
 1   Product ID:               75 non-null     object 
 2   Store Name                75 non-null     object 
 3   Product Name:             75 non-null     object 
 4   Product Price:            75 non-null     object 
 5   Total Number of Ratings:  75 non-null     object 
 6   Average Ratings:          75 non-null     float64
 7   Total Sold:               75 non-null     object 
 8   Total Fav:                75 non-null     object 
 9   Total Available:          74 non-null     object 
 10  Description:              73 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 6.6+ KB
None


In [103]:
if "Unnamed: 0" in product_table.columns:
    product_table = product_table.drop(labels="Unnamed: 0", axis=1)
    
display(product_table.head(1))

,Product ID:,Store Name,Product Name:,Product Price:,Total Number of Ratings:,Average Ratings:,Total Sold:,Total Fav:,Total Available:,Description:
0,1ca606d9-0507-410b-a98c-a0d1745cc233,Skullcandy Singapore Official Store,Crusher ANC 2 Sensory Bass Wireless Headphones...,$350.00,3,5.0,7,Favorite (43),2 pieces available,The next generation of our iconic Crusher head...


In [104]:
if "Store Name" in product_table.columns:
    merchant_table = pd.DataFrame(product_table['Store Name'])
    product_table  = product_table.drop(labels="Store Name", axis=1)

display(merchant_table.head(1))
display(product_table.head(1))

,Store Name
0,Skullcandy Singapore Official Store


,Product ID:,Product Name:,Product Price:,Total Number of Ratings:,Average Ratings:,Total Sold:,Total Fav:,Total Available:,Description:
0,1ca606d9-0507-410b-a98c-a0d1745cc233,Crusher ANC 2 Sensory Bass Wireless Headphones...,$350.00,3,5.0,7,Favorite (43),2 pieces available,The next generation of our iconic Crusher head...


In [105]:
rename_dict = {
    "Product ID:"              :"product_id",
    "Product Name:"           :"name",
    "Product Price:"          :"price",
    "Total Number of Ratings:":"total_rating",
    "Average Ratings:"        :"avg_rating",
    "Total Sold:"             :"total_sold",
    "Total Fav:"              :"fav_count",
    "Total Available:"        :"qty_avail",
    "Description:"            :"description"
}

product_table = product_table.rename(columns=rename_dict)
display(product_table.head(1))

,product_id,name,price,total_rating,avg_rating,total_sold,fav_count,qty_avail,description
0,1ca606d9-0507-410b-a98c-a0d1745cc233,Crusher ANC 2 Sensory Bass Wireless Headphones...,$350.00,3,5.0,7,Favorite (43),2 pieces available,The next generation of our iconic Crusher head...


In [106]:
def product_price_fix(x):
    if '-' in x:
        x = x.split(' - ')[0]

    x = x.strip()[1:]
    
    return float(x)
    
if product_table['price'].dtype != 'float64':
    product_table['price'] = product_table['price'].apply(product_price_fix)

print(product_table['price'].dtype)
display(product_table[['price']].head(5))

float64


,price
0,350.00
1,17.33
2,18.50
3,32.90
4,50.00


In [107]:
def product_totalRating_fix(x):
    if 'k' in x or 'K' in x:
        return int(float(x[:-1]) * 1000)
        
    elif 'm' in x or 'M' in x:
        return int(float(x[:-1]) * 1000000)

    else:
        return int(x)

if product_table['total_rating'].dtype != 'int64':
    product_table['total_rating'] = product_table['total_rating'].apply(product_totalRating_fix)

print(product_table['total_rating'].dtype)
display(product_table[['total_rating']].head(5))

int64


,total_rating
0,3
1,245
2,1600
3,210
4,100


In [108]:
def product_totalSold_fix(x):
    if 'k' in x or 'K' in x:
        return int(float(x[:-1]) * 1000)
        
    elif 'm' in x or 'M' in x:
        return int(float(x[:-1]) * 1000000)

    else:
        return int(x)

if product_table['total_sold'].dtype != 'int64':
    product_table['total_sold'] = product_table['total_sold'].apply(product_totalSold_fix)

print(product_table['total_sold'].dtype)
display(product_table[['total_sold']].head(5))

int64


,total_sold
0,7
1,813
2,5100
3,576
4,249


In [109]:
def product_favCount_fix(x):
    x = x.split(' ')[-1]
    x = x[1:-1]
    
    if 'k' in x or 'K' in x:
        return int(float(x[:-1]) * 1000)
        
    elif 'm' in x or 'M' in x:
        return int(float(x[:-1]) * 1000000)

    else:
        return int(x)

if product_table['fav_count'].dtype != 'int64':
    product_table['fav_count'] = product_table['fav_count'].apply(product_favCount_fix)

print(product_table['fav_count'].dtype)
display(product_table[['fav_count']].head(5))

int64


,fav_count
0,43
1,1100
2,1500
3,460
4,405


In [110]:
def product_qtyAvail_fix(x):
    x = x.split(' ')[0]
    return int(x)

if product_table['qty_avail'].dtype != 'int64':
    product_table['qty_avail'] = product_table['qty_avail'].fillna('0')
    product_table['qty_avail'] = product_table['qty_avail'].apply(product_qtyAvail_fix)

print(product_table['qty_avail'].dtype)
display(product_table[['qty_avail']].head(5))

int64


,qty_avail
0,2
1,2
2,477
3,100
4,8


In [124]:
print(product_table.shape)
display(product_table.head(3))
print(product_table.info())

(75, 11)


,Unnamed: 0,Product ID:,Store Name,Product Name:,Product Price:,Total Number of Ratings:,Average Ratings:,Total Sold:,Total Fav:,Total Available:,Description:
0,0,1ca606d9-0507-410b-a98c-a0d1745cc233,Skullcandy Singapore Official Store,Crusher ANC 2 Sensory Bass Wireless Headphones...,$350.00,3,5.0,7,Favorite (43),2 pieces available,The next generation of our iconic Crusher head...
1,1,a45e9ef8-c2d2-4495-ac16-0b8c42a236ca,sprise_localstore.sg,SPRISE Premium Wireless Bluetooth Earphone Col...,$17.33,245,4.6,813,Favorite (1.1k),2 pieces available,Hey~ Welcome to SPRISE Official Store! Please ...
2,2,cdba870d-eb79-4380-9de9-d6e61c895949,SHAVALIFE,USB PC Headset Conference Call with Noise Canc...,$18.50,1.6k,4.9,5.1k,Favorite (1.5k),477 pieces available,Headset Key Features: ✅ High Quality Stereo S...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                75 non-null     int64  
 1   Product ID:               75 non-null     object 
 2   Store Name                75 non-null     object 
 3   Product Name:             75 non-null     object 
 4   Product Price:            75 non-null     object 
 5   Total Number of Ratings:  75 non-null     object 
 6   Average Ratings:          75 non-null     float64
 7   Total Sold:               75 non-null     object 
 8   Total Fav:                75 non-null     object 
 9   Total Available:          74 non-null     object 
 10  Description:              73 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 6.6+ KB
None


In [139]:
product_table.to_csv("../asset/etlProduct.csv", index=False)

## #2.2 Merchant Table
1. Created from Product Table
2. Drop duplicated merchants
3. Assign unique id to each merchant

In [125]:
print(merchant_table.shape)
display(merchant_table.head(3))
print(merchant_table.info())

(75, 1)


,Store Name
0,Skullcandy Singapore Official Store
1,sprise_localstore.sg
2,SHAVALIFE


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Store Name  75 non-null     object
dtypes: object(1)
memory usage: 732.0+ bytes
None


In [137]:
merchant_table = merchant_table.drop_duplicates()
merchant_table['merchant_id'] = [uuid.uuid4() for _ in range(len(merchant_table))]

merchant_table = merchant_table.rename(columns={'Store Name':'name'})

merchant_table = merchant_table[['merchant_id','name']]
display(merchant_table.head(3))

,merchant_id,name
0,1ca5fde2-5f5d-4c9d-9c54-4cc487688774,Skullcandy Singapore Official Store
1,e101a9c3-90b7-4115-ba10-724730b52ee8,sprise_localstore.sg
2,9867ce78-67bd-4ea4-88b1-0311ed11cb46,SHAVALIFE


In [140]:
merchant_table.to_csv("../asset/etlMerchant.csv", index=False)

## #2.3 Review Table (Brendan)
1. Drop 'Unnamed: 0' column.
2. Move the review_id to the first column.
3. Rename column names according to the Data Dictionary.
4. Ensure each column has the correct data type according to the Data Dictionary.

In [113]:
# REVIEW
print(review_table.shape)
display(review_table.head(3))
print(review_table.info())

(27692, 8)


,Unnamed: 0,Product ID,Product Name,Username,Rating,DateTime,Comment ID,Comments
0,0,738907d7-39a7-403e-a8a6-1b0933baf48f,SPRISE Premium Wireless Bluetooth Earphone Col...,i*****b,5,2023-09-02 01:13,3aacfcce-8062-403d-b903-7edecee7cd1a,Best buy ever\r\nit looks great works great\r\...
1,1,738907d7-39a7-403e-a8a6-1b0933baf48f,SPRISE Premium Wireless Bluetooth Earphone Col...,jessylim70,5,2023-07-21 23:33,a34bed49-2914-4998-959f-0ba884aafcd5,Item received in good condition.\r\nBought dur...
2,2,738907d7-39a7-403e-a8a6-1b0933baf48f,SPRISE Premium Wireless Bluetooth Earphone Col...,s*****b,5,2023-05-08 19:37,53166827-9814-4518-aee9-06c23cf75b70,Value For Money: yes\r\nBest Feature(s): comfo...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27692 entries, 0 to 27691
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    27692 non-null  int64 
 1   Product ID    27692 non-null  object
 2   Product Name  27692 non-null  object
 3   Username      27692 non-null  object
 4   Rating        27692 non-null  int64 
 5   DateTime      27692 non-null  object
 6   Comment ID    12760 non-null  object
 7   Comments      12702 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.7+ MB
None


In [ ]:
review_table.to_csv("../asset/etlReview.csv", index=False)

## #2.4 User Table (Brendan)
1. Drop 'Unnamed: 0' column.
2. Rename column names according to the Data Dictionary.

In [115]:
# USER
print(user_table.shape)
display(user_table.head(3))
print(user_table.info())

(19978, 7)


,Unnamed: 0,Username,Number of Comments,Comments,Number of Products Commented,Products Commented,Mean Rating
0,0,.*****.,1,['Received but not tested if working or not'],1,{'c6248603-e4db-403f-a7d2-4139e8463d34': 1},5.0
1,1,.*****9,1,"['not the cheapest , included tool is nice but...",1,{'51f70b33-555b-4d40-ad73-a4f19446fc73': 1},5.0
2,2,.*****f,0,[nan],1,{'b17e829b-3318-49c4-8a94-edc482f7733c': 1},5.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19978 entries, 0 to 19977
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    19978 non-null  int64  
 1   Username                      19978 non-null  object 
 2   Number of Comments            19978 non-null  int64  
 3   Comments                      19978 non-null  object 
 4   Number of Products Commented  19978 non-null  int64  
 5   Products Commented            19978 non-null  object 
 6   Mean Rating                   19978 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.1+ MB
None


In [ ]:
user_table.to_csv("../asset/etlUser.csv", index=False)